In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
import matplotlib

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

Testing out the BERT model

In [7]:
model.training

False

In [40]:
tokenizer.tokenize("Elmegyek vadászni")

['Elm', '##eg', '##yek', 'vad', '##ász', '##ni']

In [42]:
with torch.no_grad():
    output=model(
        **tokenizer(
            ["Elmegyek vadászni", "Nap süt."],
            return_tensors="pt", 
            padding="longest"
        ), 
        output_hidden_states=True
    )
for on in output:
    print(len(on))

2
2
13


In [43]:
(output[2][0]).size()

torch.Size([2, 8, 768])

In [ ]:
with torch.no_grad():
    output=model(
        **tokenizer(
            ["Elmentem vadászni."],
            return_tensors="pt",
            padding="longest"
        )
    )
print(output)

Reading in the test text from the file

In [3]:


data = pd.read_csv(
    "dev.tsv",
    sep='\t',
    names=['sentence',
           'word',
           'target_position',
           'type'
          ]
)
data

,sentence,word,target_position,type
0,Kizárólag felső utasításra cselekedtünk — mond...,utasításra,2,Sub
1,Ám érdekes módon a kormány által az Országgyűl...,tájékoztatón,9,Sup
2,"A tárca fenntartja azt a javaslatát , hogy a k...",szorzó,17,Nom
3,A termelés akadályait főként a fizetőképes ker...,szabálytalanságaiban,11,Ine
4,A szakember úgy véli : világpiac áringadozásai...,szakember,1,Nom
...,...,...,...,...
199,Az oktatási tárca kabinetfőnöke ezzel szemben ...,kabinetfőnöke,3,Nom
200,Ráadásul azokat az úgynevezett hiánypótlásokat...,teljesítéseket,20,Acc
201,A Skopjéhez közeli karambolt egy norvég Kfor-k...,sebességgel,19,Ins
202,Emellett cél olyan nagy tömegközlekedési ( int...,oldalirányban,32,Ine


Def Functions

In [49]:
def text_tokenizer(text_dataframe):
    sentences = []
    tokenized_words=[]
    target_indexes=[]
    for i in range(text_dataframe.iloc[:,0].size):
        sentence, tokenized_word, target_index = sentence_tokenizer(
        text_dataframe.iloc[i,0].split(" "),
        text_dataframe.iloc[i,2]
        )
        sentences.append(sentence)
        tokenized_words.append(tokenized_word)
        target_indexes.append(target_index)
        print(target_indexes)
    text_tokenized = pd.DataFrame({
        'sentence' : sentences,
        'word' : tokenized_words,
        'target_idx' : target_indexes,
        'type' : text_dataframe.iloc[:,3],
    })
    return text_tokenized

In [44]:
def sentence_tokenizer(sentence, target_index):
    tokenized_sentence=[]
    good_target_index=target_index
    for word in sentence:
        tokenized_word=tokenizer.tokenize(word)
        tokenized_sentence.extend(tokenized_word)
        if target_index > 0:
            good_target_index += len(tokenized_word)-1
        if target_index == 0:
            tokenized_special_word=tokenized_word
        target_index -= 1
    return tokenized_sentence, tokenized_special_word, good_target_index

Test functions

In [50]:
test_df=pd.DataFrame({
    'sentence' : data.iloc[0:5,0],
    'word' : data.iloc[0:5,1],
    'target_idx' : data.iloc[0:5,2],
    'type' : data.iloc[0:5,3],
})
print(test_df.iloc[1,2])
test_df2=text_tokenizer(test_df)
test_df2

9
[5]
[5, 20]
[5, 20, 40]
[5, 20, 40, 26]
[5, 20, 40, 26, 1]


,sentence,word,target_idx,type
0,"[Ki, ##zá, ##ról, ##ag, felső, ut, ##as, ##ítá...","[ut, ##as, ##ítás, ##ra]",5,Sub
1,"[Á, ##m, é, ##rde, ##kes, módon, a, kormány, á...","[tá, ##j, ##ék, ##ozta, ##tón]",20,Sup
2,"[A, tá, ##rca, fenn, ##tar, ##tja, azt, a, ja,...","[sz, ##or, ##zó]",40,Nom
3,"[A, terme, ##lés, aka, ##dá, ##lya, ##it, főké...","[sz, ##ab, ##ály, ##talan, ##sága, ##iban]",26,Ine
4,"[A, sz, ##ake, ##mber, úgy, v, ##él, ##i, :, v...","[sz, ##ake, ##mber]",1,Nom


In [10]:
listtest2=[
    "Autó",
    "!",
    "Szia",
    "margaréta",
    "."
]
number=2
listtest2, word, number = sentence_tokenizer(listtest2, number)
print(number)
print(word)
print(listtest2)

3
['S', '##zia']
['Au', '##tó', '!', 'S', '##zia', 'mar', '##gar', '##éta', '.']


In [69]:
a=len(tokenizer.tokenize("alma"))
a

1

In [9]:
dataframetest=pd.DataFrame({
    'test1' : data.iloc[:,1],
    'imported' : data.iloc[:,2],
})
dataframetest

,test1,imported
0,utasításra,2
1,tájékoztatón,9
2,szorzó,17
3,szabálytalanságaiban,11
4,szakember,1
...,...,...
199,kabinetfőnöke,3
200,teljesítéseket,20
201,sebességgel,19
202,oldalirányban,32


In [101]:
i=0
listtest=datasentences.iloc[0][0]
for word in datasentences.iloc[:,0]:
    i+=1
    print(i)
    print(word)

1
['Kizárólag', 'felső', 'utasításra', 'cselekedtünk', '—', 'mondják', 'egybehangzóan', 'az', 'osztrák', 'külügyi', ',', 'belügyi', 'és', 'igazságügyi', 'tárca', 'illetékesei', ',', 'miután', 'a', 'múlt', 'hét', 'szerdán', 'ők', 'hajtották', 'végre', 'az', '57', 'esztendős', 'Talics', 'letartóztatását', '.']
2
['Ám', 'érdekes', 'módon', 'a', 'kormány', 'által', 'az', 'Országgyűlésnek', 'eljuttatott', 'tájékoztatón', '—', 'amely', 'már', 'tartalmazza', 'az', 'állítólag', 'pénteken', 'szétosztott', '13', 'milliárdot', 'is', '—', 'két', 'nappal', 'korábbi', ',', 'augusztus', '25-ei', 'dátum', 'szerepel', '.']
3
['A', 'tárca', 'fenntartja', 'azt', 'a', 'javaslatát', ',', 'hogy', 'a', 'közoktatás', 'területén', 'nem', 'pedagógus', 'munkakörben', 'foglalkoztatottak', 'számára', '1,12', 'szorzó', 'bevezetésére', 'kerüljön', 'sor', ',', 'és', 'azt', 'is', ',', 'hogy', 'a', 'pedagógusok', 'kiemelkedő', 'munkavégzéséért', 'járó', 'keresetkiegészítés', 'összege', 'a', 'duplájára', 'növekedjen', '

In [8]:
sentences=datasentences.sentence.values
print(type(sentences))
sentences

<class 'numpy.ndarray'>


array([list(['Kizárólag', 'felső', 'utasításra', 'cselekedtünk', '—', 'mondják', 'egybehangzóan', 'az', 'osztrák', 'külügyi', ',', 'belügyi', 'és', 'igazságügyi', 'tárca', 'illetékesei', ',', 'miután', 'a', 'múlt', 'hét', 'szerdán', 'ők', 'hajtották', 'végre', 'az', '57', 'esztendős', 'Talics', 'letartóztatását', '.']),
       list(['Ám', 'érdekes', 'módon', 'a', 'kormány', 'által', 'az', 'Országgyűlésnek', 'eljuttatott', 'tájékoztatón', '—', 'amely', 'már', 'tartalmazza', 'az', 'állítólag', 'pénteken', 'szétosztott', '13', 'milliárdot', 'is', '—', 'két', 'nappal', 'korábbi', ',', 'augusztus', '25-ei', 'dátum', 'szerepel', '.']),
       list(['A', 'tárca', 'fenntartja', 'azt', 'a', 'javaslatát', ',', 'hogy', 'a', 'közoktatás', 'területén', 'nem', 'pedagógus', 'munkakörben', 'foglalkoztatottak', 'számára', '1,12', 'szorzó', 'bevezetésére', 'kerüljön', 'sor', ',', 'és', 'azt', 'is', ',', 'hogy', 'a', 'pedagógusok', 'kiemelkedő', 'munkavégzéséért', 'járó', 'keresetkiegészítés', 'összege',

In [15]:
sen_for_model = [];
for sent in sentences:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-15-9a35d9c04806>, line 3)

In [7]:
datasentences = data.sentence.str.split(" ").to_frame()
datasentences

,sentence
0,"[Kizárólag, felső, utasításra, cselekedtünk, —..."
1,"[Ám, érdekes, módon, a, kormány, által, az, Or..."
2,"[A, tárca, fenntartja, azt, a, javaslatát, ,, ..."
3,"[A, termelés, akadályait, főként, a, fizetőkép..."
4,"[A, szakember, úgy, véli, :, világpiac, áringa..."
...,...
199,"[Az, oktatási, tárca, kabinetfőnöke, ezzel, sz..."
200,"[Ráadásul, azokat, az, úgynevezett, hiánypótlá..."
201,"[A, Skopjéhez, közeli, karambolt, egy, norvég,..."
202,"[Emellett, cél, olyan, nagy, tömegközlekedési,..."


In [21]:
tokenizer.tokenize(datasentences.iloc[0])

AttributeError: 'Series' object has no attribute 'strip'